<a href="https://colab.research.google.com/github/lyj1994/notebook/blob/main/%E7%88%AC%E8%99%AB%E4%B8%8E%E8%AF%8D%E4%BA%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 背景

这次想要做的是把《经济研究》的题目都爬取出来，然后通过中文分词后，生成一个词云图。  
把大象关进冰箱分三步，我们这个也只需要3步。  
1. 爬取数据
2. 分词
3. 生成词云

## 爬取数据

先说爬取数据，直接找到知网的经济研究详情页：https://navi.cnki.net/KNavi/JournalDetail?pcode=CJFD&pykm=JJYJ&Year=&Issue=  
打开Chrome的控制台，发现每次切换左侧的刊登月份时，都会发一个请求，形如：https://navi.cnki.net/knavi/JournalDetail/GetArticleList?year=2020&issue=08&pykm=JJYJ&pageIdx=0&pcode=CJFD ，很明显，URL中的`year`和`issue`用来控制刊物的年份和月份，`pykm`就是对应的刊物了，`pageIdx`则是用来分页的，我们先来爬一个2020年8月的刊物。  


In [1]:
# 安装需要的库和依赖
!pip install requests
!pip install beautifulsoup4
!pip install jieba
!pip install urllib2

import requests
from bs4 import BeautifulSoup

response = requests.get('https://navi.cnki.net/knavi/JournalDetail/GetArticleList?year=2020&issue=08&pykm=JJYJ&pageIdx=0&pcode=CJFD')
response.text

text = response.content.decode('utf8')
soup = BeautifulSoup(text, 'html.parser')

ERROR: Could not find a version that satisfies the requirement urllib2 (from versions: none)
ERROR: No matching distribution found for urllib2


## 处理数据&分词
我们观察到，这个返回的HTML脚本里面，使用下面的结果包裹了每一篇的标题和链接：

```html
<span class="name">
    <a target="_blank" href="Common/RedirectPage?sfield=FN&amp;dbCode=CJFD&amp;filename=JJYJ202008002&amp;tableName=CJFDAUTO&amp;url=">
        全球经济大变局、中国潜在增长率与后疫情时期高质量发展
    </a>
</span>
```

所以通过`BeautifulSoup`处理过后的脚本，我们可以通过find_all方法找到对应的span标签，然后，在for循环中，一一取出对应的href和text。
在for循环处理时，也可以顺便使用[`jieba`分词](https://github.com/fxsjy/jieba)把标题都分析出来，最后拼接成一个长字符串，用来之后生成词云。  
当然，只有一期刊物是不行的。We need more！  

## 多个刊物

如果有什么问题一次循环解决不了，那就两次。  
我们爬取2020年8月，传入的year是2020，issue是08，所以，显然，把这里year和issue变更一下就好了。
代码如下：

In [2]:
from urllib.parse import parse_qs
import jieba
import jieba.analyse
import requests
from bs4 import BeautifulSoup
import time

start_year = 2020
stop_year = 2018 # 只爬千禧年之后的数据
issue_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# 这里的排除词主要是排除一些常用但是对我们价值不大的词汇
exclude_word_list = ['基于', '综述', '启事', '学院', '教师', '笔谈', '来自', '一个', '公告', '主编', '经济', '中国'] 

essay_notice = '征文启事' # 去除征文启事这种无关标题
prefix = 'https://kns.cnki.net/kcms/detail/detail.aspx'

jieba.enable_paddle()

title_href_list = [] # 存放检索结果的list
participles = '' # 记录全部分词的长字符串

def participles_generator (year, issue, o_participles):
    t_participles = o_participles
    response = requests.get('https://navi.cnki.net/knavi/JournalDetail/GetArticleList?year=' + str(year) + '&issue='+ issue +'&pykm=JJYJ&pageIdx=0&pcode=CJFD')
    text = response.content.decode('utf8')
    soup = BeautifulSoup(text, 'html.parser')
    span_name_list = soup.find_all("span", "a", class_="name")

    for tag in span_name_list:
        title = tag.find('a').text.strip() # 去掉空白字符的影响
        if (title.find(essay_notice) == -1): # 去掉征文启事
            query = parse_qs(prefix + tag.find('a')['href'])
            # 把标题和链接先保存起来，词云生成暂不使用
            title_href_list.append([title, prefix + '?dbname' + query['dbCode'][0] + '&filename=' + query['filename'][0] + '&dbname=' + query['tableName'][0], jieba.analyse.extract_tags(title, withWeight=True)])
            # 把分词结果写入到一个长字符串里
            for x, w in jieba.analyse.extract_tags(title, withWeight=True):
                if (w > 0.5 and exclude_word_list.count(x) == 0): # 权重大于0.5才使用，去掉一些水词
                    t_participles = t_participles + ' ' + x
    return t_participles

while (start_year > stop_year):
    for idx, issue in enumerate(issue_list):
        if (start_year == 2020 and idx >= 8): # 截止编写代码之时，2020只有到8月刊
            pass
        else:
            print('当前执行月份：{}年{}月'.format(start_year, issue))
            time.sleep(5) # 每次睡5s，防止爬取太多频次被封
            participles = participles_generator(start_year, issue, participles)
    start_year = start_year - 1
participles


Installing paddle-tiny, please wait a minute......
Paddle enabled successfully......


当前执行月份：2020年01月
当前执行月份：2020年02月
当前执行月份：2020年03月
当前执行月份：2020年04月
当前执行月份：2020年05月
当前执行月份：2020年06月
当前执行月份：2020年07月
当前执行月份：2020年08月
当前执行月份：2019年01月
当前执行月份：2019年02月
当前执行月份：2019年03月
当前执行月份：2019年04月
当前执行月份：2019年05月
当前执行月份：2019年06月
当前执行月份：2019年07月
当前执行月份：2019年08月
当前执行月份：2019年09月
当前执行月份：2019年10月
当前执行月份：2019年11月
当前执行月份：2019年12月


''

## 生成词云
在处理词云数据之前，需要下载中文字体，因为一些字体收费或者不能商用等原因，选择了思源黑体，大家可以在 GitHub 上下载下来，使用 regular.ttf 即可。https://github.com/Pal3love/Source-Han-TrueType。  
类似于本地加载字体文件，在 Colab 中加载文件也需要显式的文件地址，但是 Colab 采用的容器，每次重新启动后，上一次传入文件就会销毁。所以我们需要上传到账号对应的 Google 网盘，然后在 Colab 的笔记本中加载 Google 网盘中文件，找到对应的文件以后，点击三个点，选择“复制路径”，就可以使用 Image.open 打开这个路径了。  
需要注意的是，我这里写的是我自己的路径，如果大家可以根据自己实际网盘路径吧这两个path都修改一下。

In [6]:
from wordcloud import WordCloud
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import BytesIO

# 这里换成了一张网络图片地址，也可以直接使用 Image.open('/image_path') 打开本地图片
response = requests.get("https://i.loli.net/2020/10/26/2eiPWczMoRgrxEb.png")
img = Image.open(BytesIO(response.content))

map_mask = np.array(img)

# 自行找到ttf字体文件存放的目录，然后拷贝过来
wordcloud = WordCloud(font_path="/content/drive/My Drive/ttf/SourceHanSansCN-Regular.ttf",background_color="white", max_words=2000, mask=map_mask, width=2400, height=1800, max_font_size=100).generate(participles)
%pylab inline
plt.figure(dpi=200)
plt.imshow(wordcloud, interpolation='bilinear',)
plt.axis("off")

ValueError: ignored